<a href="https://colab.research.google.com/github/carlos-alves-one/-BlockChain-Explorer-Project/blob/main/blockchain_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### BlockChain Explorer Project

# Goal No.1

Calculate the transaction ID

## Step 1: Import Libraries


In [83]:
# Import the hashlib library for secure hash and message digest algorithms
import hashlib

# Import the hashes module from cryptography for cryptographic hashing
from cryptography.hazmat.primitives import hashes

# Import default_backend for cryptographic algorithm support
from cryptography.hazmat.backends import default_backend

# Import serialization for converting keys and certificates between Python objects and bytes
from cryptography.hazmat.primitives import serialization

# Import ec (Elliptic Curve) module for elliptic curve cryptography operations
from cryptography.hazmat.primitives.asymmetric import ec

# Import InvalidSignature exception for handling signature verification failures
from cryptography.exceptions import InvalidSignature


## Step 2: Define Transaction Details
Outline how to define the transaction details, such as sender and recipient addresses, the amount, the fee, the nonce, and the digital signature. Mention that these details form the basis of the transaction and are crucial for its unique identification.

In [84]:
sender_addr      = "b66eeaa8059bd4d50c760beb93c2eb086ad91853"
recipient_addr   = "b8e6687270bade89f03178d0195dbde30be1287c"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004abb6abed08a9304fedaa394bd578fde6e28a3721ef25da5063fd3ddf"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"


## Step 3: Define functions to Hash and calculate the Transaction ID


In [85]:
# SHA1 hash calculation
def calculate_sha1_hash(public_key):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(public_key)
    return digest.finalize()

# Signature hash calculation
def calculate_signature_hash(recipient_hash, amount, fee, nonce):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(recipient_hash)
    digest.update(amount.to_bytes(8, byteorder='little', signed=False))
    digest.update(fee.to_bytes(8, byteorder='little', signed=False))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    return digest.finalize()

# Transaction  ID calculation
def calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(sender_hash)
    digest.update(recipient_hash)
    digest.update(sender_public_key)
    digest.update(amount.to_bytes(8, byteorder='little', signed=False))
    digest.update(fee.to_bytes(8, byteorder='little', signed=False))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    digest.update(signature)
    return digest.finalize()


## Step 4: Calculates the Hashes and the Transaction ID


In [86]:
# Hash the sender address
sender_hash = calculate_sha1_hash(sender_addr.encode('utf-8'))

# Hash the recipient address
recipient_hash = calculate_sha1_hash(recipient_addr.encode('utf-8'))

# Calculate digital signature
digital_signature = calculate_signature_hash(recipient_hash, amount, fee, nonce)

# Encode the sender's public key from string to bytes using UTF-8 encoding
sender_publickey = sender_publickey.encode('utf-8')

# Generate the transaction ID
tx_id_bytes = calculate_txid(sender_hash, recipient_hash, sender_publickey, amount, fee, nonce, digital_signature)

# Convert the transaction ID from bytes to a hexadecimal string
tx_id = tx_id_bytes.hex()


## Step 5: Output the Transaction ID


In [87]:
print("The calculated transaction ID is:", tx_id)


The calculated transaction ID is: fe1f2d9e1651a97e0867ff8f38a52b56163a8c20e936d0f9fcbe29d01e7b1dcb


# Goal No.2

Verify the following transactions:
- Check the sender’s address,
- Check digital signature
- Check transaction ID

## Transaction No.1

In [88]:
sender_addr      = "3e6cfd7644cc96050c506d5246ea831998a98412"
recipient_addr   = "263b26367061e7c505b520a192c5c3d079f04808"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004b7348c4c2c 97fc8d0e1645fa868d32ca4de03f0772cb76bec66d535d977177896115445298b10cd378dd66afc6bff24a7827083251dd210303d0699f327a7134"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"
transaction_id    = "24639171a1a6fe688b410764c02d6f1a103fb5a16690ceef959e7647da8deb3d"


### Verify Sender's Address

In [89]:
# Check sender's address
sender_hash    = calculate_sha1_hash(sender_addr.encode('utf-8'))
recipient_hash = calculate_sha1_hash(recipient_addr.encode('utf-8'))

# Convert the public key and digital signature to bytes
sender_publickey_bytes  = bytes.fromhex(sender_publickey)
digital_signature_bytes = bytes.fromhex(digital_signature)

# Generate the transaction ID
tx_id_bytes = calculate_txid(sender_hash, recipient_hash, sender_publickey_bytes, amount, fee, nonce, digital_signature_bytes)

# Convert the transaction ID from bytes to a hexadecimal string
tx_id = tx_id_bytes.hex()

# Compare the calculated transaction ID with the given transaction ID
if tx_id == transaction_id:
    print("\n-> The sender's address is verified")
else:
    print("\n-> The sender's address does not match the transaction details")



-> The sender's address does not match the transaction details


### Verify Digital Signature

In [90]:
# Declare function to load a public key from a hexadecimal string
def load_public_key(hex_public_key):
    public_key_bytes = bytes.fromhex(hex_public_key)
    public_key = serialization.load_der_public_key(public_key_bytes)
    return public_key

# Declare function to verify the digital signature
def verify_signature(public_key, data, signature):
    try:
        public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))
        return True
    except InvalidSignature:
        return False

# Assuming we have the sender's public key, transaction data, and the signature
sender_publickey_hex  = sender_publickey    # The provided public key in hex format
digital_signature_hex = digital_signature  # The digital signature in hex format

# Convert digital signature from hex to bytes
digital_signature_bytes = bytes.fromhex(digital_signature_hex)

# Prepare the data that was signed
data_to_verify = recipient_hash + amount.to_bytes(8, byteorder='little', signed=False) + fee.to_bytes(8, byteorder='little', signed=False) + nonce.to_bytes(8, byteorder='little', signed=False)

# Load the public key
public_key = load_public_key(sender_publickey_hex)

# Verify the signature
if verify_signature(public_key, data_to_verify, digital_signature_bytes):
    print("\n-> The digital signature is valid.")
else:
    print("\n-> The digital signature is invalid.")



-> The digital signature is invalid.


### Verify Transaction ID

In [91]:
# Convert hexadecimal values to bytes
sender_publickey_bytes  = bytes.fromhex(sender_publickey)
digital_signature_bytes = bytes.fromhex(digital_signature)

# Calculate hashes
sender_hash    = calculate_sha1_hash(sender_addr.encode('utf-8'))
recipient_hash = calculate_sha1_hash(recipient_addr.encode('utf-8'))

# Calculate the transaction ID
tx_id_bytes = calculate_txid(sender_hash, recipient_hash, sender_publickey_bytes, amount, fee, nonce, digital_signature_bytes)

# Convert the transaction ID from bytes to a hexadecimal string
calculated_tx_id = tx_id_bytes.hex()

# Compare the calculated transaction ID with the given transaction ID
if calculated_tx_id == transaction_id:
    print("\nThe transaction ID is correct.")
else:
    print(f"\nThe transaction ID is incorrect. \n\nCalculated: {calculated_tx_id} \nGiven.....: {transaction_id}")



The transaction ID is incorrect. 

Calculated: cf9d68a2181a4aff3e2659624b575505e613570576ea20e9d8cfaac0abd9be5f 
Given.....: 24639171a1a6fe688b410764c02d6f1a103fb5a16690ceef959e7647da8deb3d
